In [ ]:
!pip install opencv-python scikit-lear

In [ ]:

import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Função que vai carregar imagens de múltiplas pastas
def load_images_from_folder(base_path, image_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(base_path))
    class_to_idx = {cls: i for i, cls in enumerate(class_names)}
    
    for cls in class_names:
        class_folder = os.path.join(base_path, cls)
        if not os.path.isdir(class_folder):
            continue
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            if os.path.isfile(img_path):
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, image_size)
                    images.append(img)
                    labels.append(class_to_idx[cls])

    return np.array(images), to_categorical(labels, num_classes=len(class_names)), class_names

base_path = 'derma-hand-ai\data\doencas'

# carregando os dados com duas classes definidas
X, y, class_names = load_images_from_folder(base_path)
print(f'Classes: {class_names}')
print(f'Total de imagens: {len(X)}')

# Normalizar
X = X / 255.0

# Dividir em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# vou usar por enquanto o mobilenetV2 para treinamento
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adicionar camadas finais
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# congelando aqui camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


model.save('/content/hand_skin_model.h5')

# model.save('.\model\hand_skin_model.h5')

print("Modelo treinado e salvo com sucesso!")